In [15]:
class Reparateur:
    import notebook_importer
    from Logger      import Logger
    from Position    import Position
    nb_d_instance     = 0
    reparateur_logger = Logger("./reparateurs.csv")
    output_dir        = "./Reparateurs/"
    station_de_base   = Position(0,0)
    def __init__(self, employeur, liste_stations_pour_tournee):
        import random
        import numpy as np
        from Logger      import Logger
        from Trajet      import Trajet
        from Position    import Position
        import os

        Reparateur.nb_d_instance +=1
        
        self.id                 = Reparateur.nb_d_instance
        self.logger             = Logger(os.path.join(Reparateur.output_dir, "reparateu_{id}.csv".format(id=self.id)))
        self.employeur          = employeur
        self.travail            = Reparateur.station_de_base
        self.tournee            = list(liste_stations_pour_tournee)
        self.stations_a_reparer = list(liste_stations_pour_tournee)
        
        self.position           = self.travail
        self.trajet             = Trajet(self.travail, self.travail)
        self.positions          = [self.position]
        self.timestamp          = 0
        self.timestamps         = [self.timestamp]
        self.destination_finale = False
        self.vitesse            = 0.1
        self.wanted_details     = [ 'id', 'trajet','vitesse', 'position', 'destination_finale']
        self.status             = [{x : self.__dict__[x] for x in self.wanted_details}]
        self.log_details        = self.wanted_details
        self.logger.log(",".join(self.log_details))
        self.etait_sur_station  = False
        self.duree_reparation   = 30 
        
    def __str__(self):
        return "Reparateur %s :  %s" %(self.id,  str(self.position))
    
    def log(self):
        data    = [str(self.__dict__[attribute]) for attribute in self.log_details]
        message = ",".join(data)
        self.logger.log(message) 
        
    def get_historique(self, jitter=False):
        import pandas as pd
        df = pd.DataFrame()
        if self.positions:
            df = pd.concat([x.get_df(jitter=jitter) for x in self.positions])
            df["id_reparateur"] = self.id
            df["time"]          = self.timestamps
            
        
        return df
    def get_personal_data(self):
        import pandas as pd
        data           = [{x : self.__dict__[x] for x in self.wanted_details}]
        df             = pd.DataFrame(data)
        return df
    
    def flush_historique(self):
        self.logger.write_to_disk()
        self.positions  = []
        self.timestamps = []
        self.status     = []
    
    def get_vitesse(self):
        return self.vitesse
        
    def est_au_travail(self):
        from Position    import Position
        return Position.is_same_position(self.position,self.travail)
    
    def est_arrive(self):
        from Position    import Position
        return Position.is_same_position(self.position,self.destination_finale)
            
    def est_sur_station(self):
        return self.employeur.is_position_a_station(self.position)

    def avancer(self, timestamp, debug=False):
        from Trajet import Trajet
        from Position import Position
        if not self.trajet.fini():
            self.position = self.trajet.next_position()
        else:            
            if self.est_au_travail():
                next_station_a_reparer  = self.stations_a_reparer.pop(0)
                self.destination        = next_station_a_reparer.position 
                self.trajet             = Trajet(self.travail, self.destination, self.get_vitesse())
                self.position           = self.travail
                
            if self.est_sur_station():
                station_actuelle = self.employeur.get_station_at_position(self.position)
                self.position    = station_actuelle.position
                
                if not self.etait_sur_station:
                    self.heure_d_arrivee      = timestamp
                    self.etait_sur_station    = True
                    station_actuelle.en_panne = True
                    self.trajet               = Trajet(self.position, self.position)
                else:
                    if timestamp < self.heure_d_arrivee + self.duree_reparation :
                        # on attend pour la duree de la reparation
                        pass
                    else:                        
                        station_actuelle.reparer(timestamp)
                        self.etait_sur_station = False
                        if len(self.stations_a_reparer)> 0:
                            next_station     = self.stations_a_reparer.pop(0)
                            self.trajet      = Trajet(self.position, next_station.position, self.get_vitesse())
                        else:
                            self.trajet      = Trajet(self.position, self.travail, self.get_vitesse())
                            self.stations_a_reparer = list(self.tournee)
                    
        self.positions.append(self.position)
        self.timestamp = timestamp
        self.timestamps.append(self.timestamp)
        self.log()        
        self.status.append({x : str(self.__dict__[x]) for x in self.wanted_details})
        if 100 < len(self.timestamps):
            self.flush_historique()

         